In [1]:
!pip -q install -U ultralytics

import os
import time
import yaml
import math
import shutil
from pathlib import Path

import numpy as np
import pandas as pd
from ultralytics import YOLO

print("Ultralytics:", __import__("ultralytics").__version__)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.8 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics: 8.4.9


In [2]:
!pip -q install -U "kaggle==1.8.2"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.9/88.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.4/256.4 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 131.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.4/160.4 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.4/192.4 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.2/55.2 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.8/269.8 kB 25.8 MB/s eta 0:00:00


In [4]:
import os, json
from pathlib import Path

kaggle_dir = Path.home() / ".kaggle"
kaggle_dir.mkdir(parents=True, exist_ok=True)
dst = kaggle_dir / "kaggle.json"

sources = [Path("/content/kaggle.json"), Path("./kaggle.json")]
src = next((p for p in sources if p.exists()), None)
if src is None:
    raise FileNotFoundError("kaggle.json not found. Place it at ./kaggle.json or /content/kaggle.json")

dst.write_text(src.read_text(), encoding="utf-8")
os.chmod(dst, 0o600)

creds = json.loads(dst.read_text(encoding="utf-8"))
os.environ["KAGGLE_USERNAME"] = creds["username"]
os.environ["KAGGLE_KEY"] = creds["key"]

print("Configured:", dst)


Configured: /root/.kaggle/kaggle.json


In [5]:
from pathlib import Path

out_dir = Path("./pizza_toppings_od")
DATA_ROOT = out_dir
out_dir.mkdir(parents=True, exist_ok=True)

!kaggle datasets download -d matthiasbartolo/pizza-toppings-object-detection -p {str(out_dir)} --unzip
print("Downloaded to:", out_dir.resolve())


Dataset URL: https://www.kaggle.com/datasets/matthiasbartolo/pizza-toppings-object-detection
License(s): MIT
100% 1.10G/1.10G [00:09<00:00, 119MB/s]
100% 1.10G/1.10G [00:09<00:00, 126MB/s]
Downloaded to: /content/pizza_toppings_od


In [6]:
PIZZA_SRC_ID = 14

OUT_ROOT = Path("./pizza_only_dataset").resolve()

IMGSZ = 640
BATCH = 16
DEVICE = 0
HALF = True
EPOCHS = 30
PATIENCE = 10

MODEL_CANDIDATES = [
    "yolo26s.pt", "yolo11s.pt","yolov8s.pt"
]

WEIGHTS = {
    "recall": 0.35,
    "precision": 0.20,
    "ap50": 0.25,
    "fps": 0.20,
}


In [7]:
DATASET_ROOT = Path("pizza_toppings_od/Pizza Object Detector.v7i.yolov8").resolve()

assert DATASET_ROOT.exists(), f"Not found: {DATASET_ROOT}"

print(DATASET_ROOT)


/content/pizza_toppings_od/Pizza Object Detector.v7i.yolov8


In [8]:
def pick_val_name(root: Path):
    if (root / "valid").exists():
        return "valid"
    if (root / "val").exists():
        return "val"
    raise FileNotFoundError("Neither 'valid' nor 'val' folder exists")

VAL_NAME = pick_val_name(DATASET_ROOT)

SPLITS = {
    "train": DATASET_ROOT / "train",
    "val": DATASET_ROOT / VAL_NAME,
    "test": DATASET_ROOT / "test",
}

for s, p in SPLITS.items():
    assert (p / "images").exists(), f"Missing: {p/'images'}"
    assert (p / "labels").exists(), f"Missing: {p/'labels'}"

print("Using splits:", {k: str(v) for k, v in SPLITS.items()})


Using splits: {'train': '/content/pizza_toppings_od/Pizza Object Detector.v7i.yolov8/train', 'val': '/content/pizza_toppings_od/Pizza Object Detector.v7i.yolov8/valid', 'test': '/content/pizza_toppings_od/Pizza Object Detector.v7i.yolov8/test'}


In [9]:
def ensure_empty_dir(p: Path):
    if p.exists():
        shutil.rmtree(p)
    p.mkdir(parents=True, exist_ok=True)

ensure_empty_dir(OUT_ROOT)

for split in ["train", "val", "test"]:
    (OUT_ROOT / split / "images").mkdir(parents=True, exist_ok=True)
    (OUT_ROOT / split / "labels").mkdir(parents=True, exist_ok=True)

def convert_label_file(src_lbl: Path, dst_lbl: Path, pizza_src_id: int) -> bool:
    """
    Read YOLO label file lines: class x y w h
    Keep only class==pizza_src_id and remap to class 0.
    Returns True if at least one pizza box exists.
    """
    if not src_lbl.exists():
        return False

    kept = []
    for line in src_lbl.read_text(encoding="utf-8").splitlines():
        line = line.strip()
        if not line:
            continue
        parts = line.split()
        if len(parts) < 5:
            continue
        cls = int(float(parts[0]))
        if cls == pizza_src_id:
            parts[0] = "0"
            kept.append(" ".join(parts[:5]))

    if kept:
        dst_lbl.write_text("\n".join(kept) + "\n", encoding="utf-8")
        return True
    else:
        if dst_lbl.exists():
            dst_lbl.unlink()
        return False

stats = []
for split, split_dir in SPLITS.items():
    img_dir = split_dir / "images"
    lbl_dir = split_dir / "labels"

    imgs = []
    for ext in ("*.jpg","*.jpeg","*.png","*.bmp","*.webp"):
        imgs += list(img_dir.glob(ext))
    imgs = sorted(imgs)

    kept_imgs = 0
    kept_boxes = 0
    total_imgs = len(imgs)

    for img_path in imgs:
        lbl_path = lbl_dir / (img_path.stem + ".txt")
        out_img = OUT_ROOT / split / "images" / img_path.name
        out_lbl = OUT_ROOT / split / "labels" / (img_path.stem + ".txt")

        has_pizza = convert_label_file(lbl_path, out_lbl, PIZZA_SRC_ID)
        if has_pizza:
            shutil.copy2(img_path, out_img)
            kept_imgs += 1
            kept_boxes += len(out_lbl.read_text().splitlines())

    stats.append({"split": split, "total_imgs": total_imgs, "pizza_imgs": kept_imgs, "pizza_boxes": kept_boxes})

pd.DataFrame(stats)


,split,total_imgs,pizza_imgs,pizza_boxes
0,train,2540,2522,2686
1,val,284,279,292
2,test,283,281,313


In [10]:
pizza_yaml = {
    "path": str(OUT_ROOT),
    "train": "train/images",
    "val": "val/images",
    "test": "test/images",
    "nc": 1,
    "names": ["pizza"],
}

PIZZA_YAML = Path("./pizza_only.yaml").resolve()
PIZZA_YAML.write_text(yaml.safe_dump(pizza_yaml, sort_keys=False), encoding="utf-8")

print("Wrote:", PIZZA_YAML)
print(pizza_yaml)


Wrote: /content/pizza_only.yaml
{'path': '/content/pizza_only_dataset', 'train': 'train/images', 'val': 'val/images', 'test': 'test/images', 'nc': 1, 'names': ['pizza']}


In [11]:
def normalize_series(s: pd.Series) -> pd.Series:
    s = s.astype(float)
    if s.isna().all():
        return pd.Series([0.0] * len(s), index=s.index)
    vmin, vmax = np.nanmin(s.values), np.nanmax(s.values)
    if math.isclose(vmin, vmax):
        return pd.Series([0.0] * len(s), index=s.index)
    return (s - vmin) / (vmax - vmin)

rows = []

for w in MODEL_CANDIDATES:
    print(f"\n==============================")
    print(f"TRAIN (1-class pizza): {w}")
    print(f"==============================")

    model = YOLO(w)

    train_res = model.train(
        data=str(PIZZA_YAML),
        imgsz=IMGSZ,
        batch=BATCH,
        device=DEVICE,
        epochs=EPOCHS,
        patience=PATIENCE,
        pretrained=True,
        verbose=False
    )

    best_pt = Path(train_res.save_dir) / "weights" / "best.pt"
    if not best_pt.exists():
        raise FileNotFoundError(f"best.pt not found at {best_pt}")

    print(f"\nTEST EVAL: {best_pt}")
    model_ft = YOLO(str(best_pt))

    res = model_ft.val(
        data=str(PIZZA_YAML),
        split="test",
        imgsz=IMGSZ,
        batch=BATCH,
        device=DEVICE,
        half=HALF,
        verbose=False,
        plots=False
    )

    precision = float(res.box.mp)
    recall    = float(res.box.mr)
    ap50      = float(res.box.map50)
    ap5095    = float(res.box.map)

    speed = getattr(res, "speed", {}) or {}
    inf_ms = float(speed.get("inference", np.nan))
    fps = (1000.0 / inf_ms) if (inf_ms and inf_ms > 0) else np.nan

    rows.append({
        "base_model": w,
        "best_weights": str(best_pt),
        "precision": precision,
        "recall": recall,
        "ap50": ap50,
        "ap5095": ap5095,
        "inf_ms/img": inf_ms,
        "fps_est": fps,
    })

df = pd.DataFrame(rows)
df



TRAIN (1-class pizza): yolo26s.pt
Ultralytics 8.4.9 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/pizza_only.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo26s.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, 

,base_model,best_weights,precision,recall,ap50,ap5095,inf_ms/img,fps_est
0,yolo26s.pt,/content/runs/detect/train/weights/best.pt,0.995441,0.916933,0.953185,0.928797,88.062839,11.355528
1,yolo11s.pt,/content/runs/detect/train2/weights/best.pt,0.969180,0.926518,0.950701,0.910976,5.996221,166.771718
2,yolov8s.pt,/content/runs/detect/train3/weights/best.pt,0.983059,0.926986,0.955433,0.917276,7.625015,131.147279


In [12]:
df_scored = df.copy()

df_scored["precision_n"] = normalize_series(df_scored["precision"])
df_scored["recall_n"]    = normalize_series(df_scored["recall"])
df_scored["ap50_n"]      = normalize_series(df_scored["ap50"])
df_scored["fps_n"]       = normalize_series(df_scored["fps_est"])

df_scored["score"] = (
    WEIGHTS["recall"]    * df_scored["recall_n"] +
    WEIGHTS["precision"] * df_scored["precision_n"] +
    WEIGHTS["ap50"]      * df_scored["ap50_n"] +
    WEIGHTS["fps"]       * df_scored["fps_n"]
)

cols = ["base_model","score","recall","precision","ap50","ap5095","inf_ms/img","fps_est","best_weights"]
df_ranked = df_scored.sort_values("score", ascending=False).reset_index(drop=True)
df_ranked[cols]


,base_model,score,recall,precision,ap50,ap5095,inf_ms/img,fps_est,best_weights
0,yolov8s.pt,0.859858,0.926986,0.983059,0.955433,0.917276,7.625015,131.147279,/content/runs/detect/train3/weights/best.pt
1,yolo11s.pt,0.533679,0.926518,0.969180,0.950701,0.910976,5.996221,166.771718,/content/runs/detect/train2/weights/best.pt
2,yolo26s.pt,0.331248,0.916933,0.995441,0.953185,0.928797,88.062839,11.355528,/content/runs/detect/train/weights/best.pt


YOLOv8s продемонстрировала наилучший баланс метрик:

Recall ≈ 0.93 — минимальное число пропусков, что критично для задачи подсчёта;

Precision ≈ 0.98 — низкое число ложных срабатываний;

mAP@0.5 ≈ 0.96 — высокая точность локализации объектов.

YOLO11s показала сопоставимое качество, однако немного уступает YOLOv8s по precision и mAP.

YOLO26s имеет высокие значения precision и mAP, однако её выигрыш по качеству не компенсирует значительное падение производительности.

In [13]:
from ultralytics import YOLO
from pathlib import Path

BEST_PT = Path("/content/runs/detect/train3/weights/best.pt").resolve()
assert BEST_PT.exists(), f"Model not found: {BEST_PT}"

model = YOLO(str(BEST_PT))

onnx_path = model.export(
    format="onnx",
    imgsz=640,
    opset=17,
    simplify=True,
    dynamic=False
)

print("ONNX model saved to:", onnx_path)


Ultralytics 8.4.9 🚀 Python-3.12.12 torch-2.9.0+cu126 CPU (Intel Xeon CPU @ 2.00GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/
Model summary (fused): 73 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs

PyTorch: starting from '/content/runs/detect/train3/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (21.5 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0,<2.0.0', 'onnxslim>=0.1.71', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
Using Python 3.12.12 environment at: /usr
Resolved 14 packages in 258ms
Prepared 6 packages in 6.12s
Installed 6 packages in 290ms
 + colorama==0.4.6
 + coloredlogs==15.0.1
 + humanfriendly==10.0
 + onnx==1.20.1
 + onnxruntime-gpu==1.23.2
 + onnxslim==0.1.82

requirements: AutoUpdate success ✅ 7.3s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


ONNX: starting 

In [20]:
results = model.predict("pizzsd.PNG", conf=0.25, verbose=False)

for box in results[0].boxes:
    print(int(box.cls.item()), box.conf.item())